In [3]:
import sqlalchemy as sqla
import pymysql
import pandas as pd

In [4]:
username = "user"
password = "grad5100user"
awsresource = "database-1.cwvjklnp4wu3.us-east-1.rds.amazonaws.com"
port = 3306
dbname = "topChef"
dburi = f"mysql+pymysql://{username}:{password}@{awsresource}:{port}/{dbname}"
engine = sqla.create_engine(dburi)

def df_select(sql, engine):
    "send sql to database given by engine and return the result as a pandas dataframe"
    with engine.connect() as conn:
        result = pd.read_sql(sqla.text(sql), con=conn)
    print(f"Retrieved {result.shape[0]} records")
    return result

#### 1. There are two types of top Chef programs; the ‘Masters’ programs and the regular series. The database contains information on both. We are not interested in the ‘Masters’. There is also a season called “Canada 6” which is part of the Masters series. Create versions of the chefdetails and judges tables that exclude any records coming from seasons that include the word Masters or Canada.

In [5]:
# 'Masters' programs in chefdetails
masters_q_cd = "SELECT DISTINCT season FROM chefdetails WHERE season LIKE '%Masters%';"
masters_result_cd = df_select(masters_q_cd, engine)
print("Masters programs check in chefdetails:")
print(masters_result_cd)

# 'Canada 6' in chefdetails
canada_6_q_cd = "SELECT DISTINCT season FROM chefdetails WHERE season LIKE '%Canada%';"
canada_6_result_cd = df_select(canada_6_q_cd, engine)
print("\nCanada 6 check in chefdetails:")
print(canada_6_result_cd)

# 'Masters' programs in judges
masters_q_j = "SELECT DISTINCT season FROM judges WHERE season LIKE '%Masters%';"
masters_result_judges = df_select(masters_q_j, engine)
print("\nMasters programs check in judges:")
print(masters_result_judges)

# 'Canada 6' in judges
canada_6_q_j = "SELECT DISTINCT season FROM judges WHERE season LIKE '%Canada%';"
canada_6_result_j = df_select(canada_6_q_j, engine)
print("\nCanada 6 check in judges:")
print(canada_6_result_j)

Retrieved 5 records
Masters programs check in chefdetails:
             season
0  Masters Season 1
1  Masters Season 2
2  Masters Season 3
3  Masters Season 4
4  Masters Season 5
Retrieved 1 records

Canada 6 check in chefdetails:
     season
0  Canada 6
Retrieved 5 records

Masters programs check in judges:
             season
0  Masters Season 1
1  Masters Season 2
2  Masters Season 3
3  Masters Season 4
4  Masters Season 5
Retrieved 1 records

Canada 6 check in judges:
     season
0  Canada 6


In [6]:
# Create a version of the chefdetails table that excludes records from seasons with the words 'Masters' or 'Canada'
chefdetails_q = """SELECT * FROM chefdetails 
WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';"""

chefdetails = df_select(chefdetails_q, engine)

# Create a version of the judges table that excludes records from seasons with the words 'Masters' or 'Canada'
judges_q = """SELECT * FROM judges 
WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';"""

judges = df_select(judges_q, engine)

display(chefdetails.head(), judges.head())

Retrieved 322 records
Retrieved 633 records


,num,name,chef,hometown,city,state,age,season,seasonNumber,series,placement,personOfColor,occupation,gender
0,0,Richard Blais,Richard B.,None,None,None,38.0,All Stars: New York,8,US,1,None,Owner,Male
1,1,Mike Isabella,Mike I.,None,None,None,35.0,All Stars: New York,8,US,2,None,Executive Chef/Owner,Male
2,2,Antonia Lofaso,Antonia L.,None,None,None,34.0,All Stars: New York,8,US,3,None,Executive Chef,Female
3,3,Tiffany Derry,Tiffany D.,None,None,None,27.0,All Stars: New York,8,US,4,POC,Executive Chef,Female
4,4,Carla Hall,Carla H.,None,None,None,46.0,All Stars: New York,8,US,5,POC,Chef/Owner,Female


,num,season,seasonNumber,series,episode,challengeType,outcomeType,guestJudge,competedOnTC,otherShows
0,0,All Stars: New York,8,US,1,Quickfire,Team,Tom Colicchio,None,Top Chef Judge
1,1,All Stars: New York,8,US,1,Elimination,Individual,Anthony Bourdain,None,Parts Unknown; No Reservations
2,2,All Stars: New York,8,US,2,Quickfire,Individual,Joe Jonas,None,None
3,3,All Stars: New York,8,US,2,Elimination,Team,Katie Lee,None,Beat Bobby Flay Judge
4,4,All Stars: New York,8,US,3,Quickfire,Team,David Chang,None,None


In [7]:
display(chefdetails.info(), judges.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num            322 non-null    int64  
 1   name           322 non-null    object 
 2   chef           322 non-null    object 
 3   hometown       200 non-null    object 
 4   city           303 non-null    object 
 5   state          304 non-null    object 
 6   age            219 non-null    float64
 7   season         322 non-null    object 
 8   seasonNumber   322 non-null    int64  
 9   series         322 non-null    object 
 10  placement      322 non-null    int64  
 11  personOfColor  122 non-null    object 
 12  occupation     183 non-null    object 
 13  gender         322 non-null    object 
dtypes: float64(1), int64(3), object(10)
memory usage: 35.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633 entries, 0 to 632
Data columns (total 10 columns):
 #   Column         No

None

None

Of particular interest in the chefdetails table are the fields:
- **season** which is the name of the season (often a city where the competition took place, but not always)
- **seasonNumber** which is the number of the season
- **name** which is the chef’s name
- **placement** which tells where the chef finished in the competition
- **gender** which tells the chef’s gender.

In the judges table we are interested in:
- **season** and **seasonNumber** which are the same as in the chefdetails table.
- **episode** which gives the episode where the judge appeared; the show uses many guest judges varying episode by episode.
- **challengeType** describes the type of challenge where the judge played a role.
- **guestJudge** is the judge’s name
- **competedOnTC** is ‘Yes’ if the judge was a previous top chef contestant.


### 2. Further trim your chefdetails and judges tables by including only the columns of interest from the table descriptions above.

In [8]:
chefdetails_q = """
    SELECT season, seasonNumber, name, placement, gender
    FROM chefdetails
    WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';
"""
chefdetails = df_select(chefdetails_q, engine)
display(chefdetails.info(), chefdetails.head())

Retrieved 322 records
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   season        322 non-null    object
 1   seasonNumber  322 non-null    int64 
 2   name          322 non-null    object
 3   placement     322 non-null    int64 
 4   gender        322 non-null    object
dtypes: int64(2), object(3)
memory usage: 12.7+ KB


None

,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female


In [9]:
judges_q2 = """
    SELECT season, seasonNumber, episode, challengeType, guestJudge, competedOnTC
    FROM judges
    WHERE season NOT LIKE '%Masters%' AND season NOT LIKE '%Canada%';
"""
judges = df_select(judges_q2, engine)
display(judges.info(), judges.head())

Retrieved 633 records
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633 entries, 0 to 632
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   season         633 non-null    object
 1   seasonNumber   633 non-null    int64 
 2   episode        633 non-null    int64 
 3   challengeType  633 non-null    object
 4   guestJudge     633 non-null    object
 5   competedOnTC   80 non-null     object
dtypes: int64(2), object(4)
memory usage: 29.8+ KB


None

,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None


### 3. Your tables should have information from 20 remaining different seasons, numbered from 1 to 20. Make a table that has two columns, one called season and one called seasonNumber showing the number associated with each season. (For example, San Francisco is season 1.)

In [10]:
display(chefdetails.head(), judges.head())

,season,seasonNumber,name,placement,gender
0,All Stars: New York,8,Richard Blais,1,Male
1,All Stars: New York,8,Mike Isabella,2,Male
2,All Stars: New York,8,Antonia Lofaso,3,Female
3,All Stars: New York,8,Tiffany Derry,4,Female
4,All Stars: New York,8,Carla Hall,5,Female


,season,seasonNumber,episode,challengeType,guestJudge,competedOnTC
0,All Stars: New York,8,1,Quickfire,Tom Colicchio,None
1,All Stars: New York,8,1,Elimination,Anthony Bourdain,None
2,All Stars: New York,8,2,Quickfire,Joe Jonas,None
3,All Stars: New York,8,2,Elimination,Katie Lee,None
4,All Stars: New York,8,3,Quickfire,David Chang,None


In [11]:
display(chefdetails.info(), judges.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   season        322 non-null    object
 1   seasonNumber  322 non-null    int64 
 2   name          322 non-null    object
 3   placement     322 non-null    int64 
 4   gender        322 non-null    object
dtypes: int64(2), object(3)
memory usage: 12.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633 entries, 0 to 632
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   season         633 non-null    object
 1   seasonNumber   633 non-null    int64 
 2   episode        633 non-null    int64 
 3   challengeType  633 non-null    object
 4   guestJudge     633 non-null    object
 5   competedOnTC   80 non-null     object
dtypes: int64(2), object(4)
memory usage: 29.8+ KB


None

None

In [12]:
seasons_table = chefdetails[['season', 'seasonNumber']].drop_duplicates()
seasons_table.reset_index(drop=True, inplace=True)
seasons_table.sort_values(by='seasonNumber', inplace=True)
print(seasons_table)

                 season  seasonNumber
16        San Francisco             1
11          Los Angeles             2
12                Miami             3
5               Chicago             4
14             New York             5
10            Las Vegas             6
6                  D.C.             7
0   All Stars: New York             8
18                Texas             9
17              Seattle            10
13          New Orleans            11
2                Boston            12
3            California            13
4            Charleston            14
7              Colorado            15
9              Kentucky            16
1        All-Stars L.A.            17
15             Portland            18
8               Houston            19
19      World All Stars            20


### 4. Answer the following questions using your tables so far. <br/>
#### a. Among all chef contestants, how many are male and how many are female?

In [13]:
chefdetails.shape

(322, 5)

In [14]:
# drop duplicates for a - c
chefdetails_dup = chefdetails[chefdetails[['name','gender']].duplicated() == True]
print(chefdetails_dup.shape)


chefdetails_drop = chefdetails.drop_duplicates(subset=['name', 'gender'])
print(chefdetails_drop.shape)

(50, 5)
(272, 5)


In [15]:
chefdetails.gender.value_counts(), chefdetails_drop.gender.value_counts()

(Male      170
 Female    152
 Name: gender, dtype: int64,
 Male      144
 Female    128
 Name: gender, dtype: int64)

Before I drop the duplicates, I get ***170 male, 152 female***.<br>
After dropping the duplicates when name and gender are duplicated, I get ***144 male, 128 female***.

#### b. Among all winners (placement==1) how many are male and how many are female?

In [16]:
placement1 = chefdetails[chefdetails['placement'] == 1]
placement1.gender.value_counts()

Male      14
Female     6
Name: gender, dtype: int64

In [17]:
# using dropped dataframe
placement1_drop = chefdetails_drop[chefdetails_drop['placement'] == 1]
placement1_drop.gender.value_counts()

Male      13
Female     6
Name: gender, dtype: int64

With the original data, there are ***14 male, 6 female*** as winners.<br>
With the dropped data, there are ***13 male, 6 female*** as winners.

#### c. Among all sets of top 3 finalists (placement=1,2,3) how many are male and how many are female?

In [23]:
placement1_3 = chefdetails[chefdetails['placement'] <= 3]
placement1_3.gender.value_counts()

Male      36
Female    25
Name: gender, dtype: int64

***36 male, 25 female*** are in the top 3 finalists (placement=1,2,3)

#### d. Compare the number of times a female was placed in the top 3 with the number of times a female won. Does this number seem unusual? That is, are women who place in the top 3 less likely than expected to ultimately win the competition? Why or why not?

I have no background on this show but given the question, the hypothesis would be based on a comparison of proportions.<br>

Null Hypothesis (H0):
There is no difference in the proportion of female contestants who win the competition (female winners) compared to the proportion of females in the rest of the top 3 (excluding the winner).<br>

Alternative Hypothesis (H1):
There is a difference in the proportion of female contestants who win the competition (female winners) compared to the proportion of females in the rest of the top 3 (excluding the winner).<br>

In [55]:
placement_cnt = chefdetails.groupby(['placement', 'gender']).size().unstack(fill_value=0)
placement_cnt

gender,Female,Male
placement,,
1,6,14
2,15,14
3,4,8
4,11,9
5,6,13
6,6,14
7,12,8
8,11,9
9,5,15


In [56]:
f_top3 = placement_cnt['Female'][[1, 2, 3]].sum()
f_winners = placement_cnt['Female'][1]

print("Number of females in the top 3:", f_top3)
print("Number of female winners:", f_winners)

ratio = f_winners / f_top3
print("Ratio of female winners among females placed in the top 3:", ratio)

Number of females in the top 3: 25
Number of female winners: 6
Ratio of female winners among females placed in the top 3: 0.24


In [57]:
import numpy as np
from scipy.stats import chi2_contingency

alpha = 0.05

# expected frequencies
total_females = f_top3
expected_winners = f_winners
expected_rest_top3 = total_females - expected_winners

# the observed data matrix
observed_data = np.array([[f_winners, expected_rest_top3]])

# chi-square test
chi2, p, _, _ = chi2_contingency(observed_data)

# Print results
print("Observed Data:")
print(observed_data)

print("\nExpected Frequencies:")
print("Expected Winners:", expected_winners)
print("Expected Rest of Top 3:", expected_rest_top3)

print("\nChi-Square Statistic:", chi2)
print("P-Value:", p)

# Make a decision
if p < alpha:
    print("\nReject the null hypothesis. There is a significant difference.")
else:
    print("\nFail to reject the null hypothesis. There is no significant difference.")


Observed Data:
[[ 6 19]]

Expected Frequencies:
Expected Winners: 6
Expected Rest of Top 3: 19

Chi-Square Statistic: 0.0
P-Value: 1.0

Fail to reject the null hypothesis. There is no significant difference.


### 4. Make a table with one row for each first place winner containing the winner’s name, the season they won, and a column containing ‘Yes’ or ‘No’ depending on whether they served as a guest judge or not.

In [58]:
chefdetails.columns, judges.columns

(Index(['season', 'seasonNumber', 'name', 'placement', 'gender'], dtype='object'),
 Index(['season', 'seasonNumber', 'episode', 'challengeType', 'guestJudge',
        'competedOnTC'],
       dtype='object'))

In [59]:
chefdetails_merge_4 = placement1[['name', 'season']]

In [60]:
judges_merge_4 = judges[['guestJudge', 'competedOnTC']]

In [61]:
# merge data
merged_data = pd.merge(chefdetails_merge_4, judges_merge_4, left_on='name', right_on='guestJudge', how='left').drop_duplicates()
# NaN to "No"
merged_data['competedOnTC'].fillna('No', inplace=True)

# drop 'guestJudge' column
merged_data.drop('guestJudge', axis=1, inplace=True)
merged_data

,name,season,competedOnTC
0,Richard Blais,All Stars: New York,Yes
7,Melissa King,All-Stars L.A.,Yes
10,Mei Lin,Boston,No
11,Jeremy Ford,California,No
12,Brooke Williamson,Charleston,Yes
20,Stephanie Izard,Chicago,Yes
25,Kevin Sbraga,D.C.,No
26,Joseph Flamm,Colorado,No
27,Kah-wai Lo,Houston,No
28,Kelsey Barnard Clark,Kentucky,Yes


### 5. Find all contestants who were not first place winners but did serve as a guest judge at some point. Make sure to only list each name once.

In [62]:
query = """ SELECT DISTINCT
                cd.name,
                COALESCE(CASE WHEN j.guestJudge IS NOT NULL THEN 'yes' ELSE 'no' END, 'no') AS guestJudge
            FROM
                chefdetails cd
            LEFT JOIN
                judges j ON cd.name = j.guestJudge
            WHERE
                cd.placement != 1
                AND cd.season NOT LIKE '%Masters%'
                AND cd.season NOT LIKE '%Canada 6%'
            ORDER BY
                cd.seasonNumber;
            ;"""
            
df5 = df_select(query, engine)

Retrieved 255 records


In [63]:
df5 = df5[df5["guestJudge"] == "yes"]["name"].reset_index(drop = True)
df5

0         Miguel Morales
1        Michael Midgley
2           Betty Fraser
3     Josie Smith-Malave
4          Richard Blais
5         Stefan Richter
6          Mike Isabella
7         Antonia Lofaso
8          Tiffany Derry
9             Dale Talde
10      Spike Mendelsohn
11      Stephen Asprinio
12            Edward Lee
13      Nyesha Arrington
14     Brooke Williamson
15          Nina Compton
16         Carlos Gaytan
17          Melissa King
18          Amar Santana
19        Kwame Onwuachi
20        Sheldon Simeon
21            Sam Talbot
22     Adrienne Cheatham
23          Carrie Baird
24       Bryan Voltaggio
25       Gregory Gourdet
26       Karen Akunowicz
27         Lee Anne Wong
28           Nini Nguyen
29          Dawn Burrell
30        Shota Nakajima
31          Nicole Gomes
Name: name, dtype: object